<a href="https://colab.research.google.com/github/AdiSk325/Music_ML_analyzing/blob/master/Music_Data_Augumentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Aby zacząć pracę w Colab należy podłączyć Google Drive i wykonać poniższą komórkę

In [1]:
!pip install soundfile

In [2]:
cd "/content/drive/My Drive/GIT/Music_ML_analyzing/"

/content/drive/My Drive/GIT/Music_ML_analyzing


In [0]:
import os
import pandas as pd
import numpy as np

import soundfile as sf
import librosa

In [0]:
def transpose_wav_file(path_input, path_output, n_steps):
  x, sr = librosa.load(path_input)
  y = librosa.effects.pitch_shift(x, sr, n_steps=n_steps)
  sf.write(path_output, y, sr, 'PCM_16')
  #print(f'File {path_input} was transposed to file {path_output}')
  return None
    
def chtime_wav_file(path_input, path_output, rate):
                           
  x, sr = librosa.load(path_input)
  y = librosa.effects.time_stretch(x, rate)
  sf.write(path_output, y, sr, 'PCM_16')
  #print(f'File {path_input} was chtime to file {path_output}')
        
  return None

In [0]:
HOME_DIR = '.'
DATA_DIR = os.path.join(HOME_DIR, 'data')
INPUT_DIR = DATA_DIR
OUTPUT_DIR = os.path.join(HOME_DIR,'augument_data')

In [6]:
print(os.path.realpath(INPUT_DIR))
print(os.path.realpath(OUTPUT_DIR))

/content/drive/My Drive/GIT/Music_ML_analyzing/data
/content/drive/My Drive/GIT/Music_ML_analyzing/augument_data


In [0]:
def clone_input_dir_structure(INPUT_DIR, OUTPUT_DIR):
    for dirpath, dirnames, filenames in os.walk(INPUT_DIR):
        structure = os.path.realpath(os.path.join(OUTPUT_DIR, dirpath[len(INPUT_DIR)+1:]))
        if not os.path.isdir(structure):
            os.mkdir(structure)
        else:
            print(f"{structure}\nThis folder does already exits! \n ----------------------------------------------- \n")

In [0]:
clone_input_dir_structure(INPUT_DIR, OUTPUT_DIR)

In [0]:
def create_new_data_from_raw_input(INPUT_DIR, OUTPUT_DIR):
    for dirpath, dirnames, filenames in os.walk(INPUT_DIR):
            for filename in filenames: 
                if '.wav' in filename:
                    output_path = os.path.join(OUTPUT_DIR, dirpath[len(INPUT_DIR)+1:], filename)
                    transpose_wav_file(path_input = os.path.join(dirpath, filename),
                                               path_output = output_path,
                                               n_steps=0)
                    

In [0]:
create_new_data_from_raw_input(INPUT_DIR, OUTPUT_DIR)

In [0]:
def generate_transpose_matrix(labels):
    result_mtx = pd.DataFrame(labels)
    for idx, label in enumerate(labels):
        labels.append(labels.pop(0))
        result_mtx[idx+1] = pd.Series(labels)
    return result_mtx

In [0]:
if not os.path.exists(os.path.join(OUTPUT_DIR, 'labels')):
    os.mkdir(os.path.join(OUTPUT_DIR, 'labels'))

for label in ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']:
    if not os.path.exists(os.path.join(OUTPUT_DIR, 'labels', label)):
        os.mkdir(os.path.join(OUTPUT_DIR, 'labels', label))

Transponujemy nagrania do odpowiednich tonacji

In [0]:
labels = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
transpose_matrix = generate_transpose_matrix(labels)

for dirpath, dirnames, filenames in os.walk(os.path.join(OUTPUT_DIR, 'chords', 'major')):
        for filename in filenames:
            
            filename_tonation = filename.split('.')[0]
            transpose_row = transpose_matrix[transpose_matrix[0] == filename_tonation]
            
            for step in transpose_row.columns:
                output_path = os.path.join(OUTPUT_DIR, 'labels', transpose_row[step].values[0], (dirpath.split(os.path.sep))[-2] + '_' + (dirpath.split(os.path.sep))[-1] + '_from_{}'.format(filename_tonation) + '.wav' )
                transpose_wav_file(path_input = os.path.join(dirpath, filename),
                                           path_output= output_path, 
                                           n_steps=step)                

Generujemy dane w różnych tempach

In [0]:
for dirpath, dirnames, filenames in os.walk(os.path.join(OUTPUT_DIR, 'labels')):
    for filename in filenames:
        
        filename_wo_wav = filename.split('.')[0]
                
        for time_rate in np.linspace(0.75, 1.25, 11):
            
            output_path = os.path.join(dirpath, filename_wo_wav + '_{}'.format(str(time_rate).replace('.','_')) + '.wav')
            chtime_wav_file(path_input = os.path.join(dirpath, filename),
                                    path_output= output_path,
                                    rate = time_rate)                